In [1]:
import pandas as pd
import datetime as dt

In [2]:
#reading in CSV, assigning to 'stock_data' and checking dataframe

stock_data = pd.read_csv('./stock_data/stocks_latest/stock_prices_latest.csv')
stock_data.head()

FileNotFoundError: [Errno 2] No such file or directory: './stock_data/stocks_latest/stock_prices_latest.csv'

In [3]:
#Converting date to datetime then filtering down just to stock prices from 2016 on

stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data_trim = stock_data[(stock_data['date'] > '2015-12-31')]
stock_data_trim.head()

NameError: name 'stock_data' is not defined

In [31]:
#confirming that the new dataframe is smaller than the old
print(stock_data.shape)
print(stock_data_trim.shape)

(23528435, 9)
(7650832, 9)


In [25]:
#reading in company descriptions, assigning to 'company_data' and checking dataframe
company_data = pd.read_csv('./S&P500-Info_Twitter_link.csv')

#dropping unnamed column and checking dataframe
company_data = company_data.drop(['Unnamed: 0'], axis =1)
company_data.head()

,symbol,security,sec filings,gics sector,gics sub-industry,headquarters location,date first added,cik,founded,twitter handle
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902,3MNews
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888,abbottglobal
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888),AbbvieUS
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981,Abiomedimpella
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989,Accenture


In [35]:
#merging stock and company description dataframes, only keeping S&P 500 company stock data
company_stock_data = stock_data_trim.merge(company_data, on = 'symbol')

#checking to make sure that all others have been removed
company_stock_data['security'].isna().sum()

0

In [37]:
#Exporting trimmed file
company_stock_data.to_csv(r'\stock_prices_trimmed.csv', index = False)

In [ ]:
# adding the tweets
